In [57]:
# required imports

import pandas as pd
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from numpy import savetxt

In [2]:
ds = pd.read_csv("/home/tejsax/Downloads/train.csv")
ds

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
df = ds.copy()
df.drop(["PassengerId", "Name", "Ticket", "Fare", "Cabin"], axis = 1, inplace = True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S
...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,S
887,1,1,female,19.0,0,0,S
888,0,3,female,NaN,1,2,S
889,1,1,male,26.0,0,0,C


In [4]:
def femb(s):
    if s == 'S':
        return 0
    elif s == 'C':
        return 1
    else:
        return 2

In [5]:
df["Embrked"] = df["Embarked"].apply(femb)
df.drop(["Embarked"], axis = 1, inplace = True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Embrked
0,0,3,male,22.0,1,0,0
1,1,1,female,38.0,1,0,1
2,1,3,female,26.0,0,0,0
3,1,1,female,35.0,1,0,0
4,0,3,male,35.0,0,0,0
...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,0
887,1,1,female,19.0,0,0,0
888,0,3,female,NaN,1,2,0
889,1,1,male,26.0,0,0,1


In [6]:
def fsex(str):
    if len(str) == 4:
        return 1
    else:
        return 0

In [7]:
df["sex"] = df["Sex"].apply(fsex)
df.drop(["Sex"], axis = 1, inplace = True)
df

,Survived,Pclass,Age,SibSp,Parch,Embrked,sex
0,0,3,22.0,1,0,0,1
1,1,1,38.0,1,0,1,0
2,1,3,26.0,0,0,0,0
3,1,1,35.0,1,0,0,0
4,0,3,35.0,0,0,0,1
...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,0,1
887,1,1,19.0,0,0,0,0
888,0,3,NaN,1,2,0,0
889,1,1,26.0,0,0,1,1


In [8]:
df.fillna(df.median(), inplace=True)
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Embrked,sex
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.361582,0.523008,0.381594,0.365881,0.647587
std,0.486592,0.836071,13.019697,1.102743,0.806057,0.640157,0.477990
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,3.000000,28.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,3.000000,35.000000,1.000000,0.000000,1.000000,1.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,2.000000,1.000000


In [9]:
x = df.drop(["Survived"], axis = 1)
print(x)
x = x.to_numpy()
x

     Pclass   Age  SibSp  Parch  Embrked  sex
0         3  22.0      1      0        0    1
1         1  38.0      1      0        1    0
2         3  26.0      0      0        0    0
3         1  35.0      1      0        0    0
4         3  35.0      0      0        0    1
..      ...   ...    ...    ...      ...  ...
886       2  27.0      0      0        0    1
887       1  19.0      0      0        0    0
888       3  28.0      1      2        0    0
889       1  26.0      0      0        1    1
890       3  32.0      0      0        2    1

[891 rows x 6 columns]


array([[ 3., 22.,  1.,  0.,  0.,  1.],
       [ 1., 38.,  1.,  0.,  1.,  0.],
       [ 3., 26.,  0.,  0.,  0.,  0.],
       ...,
       [ 3., 28.,  1.,  2.,  0.,  0.],
       [ 1., 26.,  0.,  0.,  1.,  1.],
       [ 3., 32.,  0.,  0.,  2.,  1.]])

In [10]:
y = df["Survived"]
print(y)
y = y.to_numpy()
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64


array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

In [11]:
# knn finding optimal k

x_axis = []
y_axis = []

for i in range(1, 50, 2):
    alg = KNeighborsClassifier(n_neighbors = i)
    score = cross_val_score(alg, x, y, cv=KFold(5, True, 0))
    x_axis.append(i)
    y_axis.append(score.mean())

x_axis, y_axis

([1,
  3,
  5,
  7,
  9,
  11,
  13,
  15,
  17,
  19,
  21,
  23,
  25,
  27,
  29,
  31,
  33,
  35,
  37,
  39,
  41,
  43,
  45,
  47,
  49],
 [0.7294896742200742,
  0.7766241918272552,
  0.7598330299416233,
  0.7721988575732849,
  0.760962902517105,
  0.7598581382210784,
  0.764333689033959,
  0.754252714832716,
  0.7463624380139351,
  0.7486347373046263,
  0.7474985876592806,
  0.7340279957315925,
  0.7171803402171866,
  0.7205260184545853,
  0.707061703596761,
  0.7048333437951164,
  0.6891406691356475,
  0.6846525641830393,
  0.677898437009604,
  0.6835101374678301,
  0.6823802648923483,
  0.675651245998368,
  0.6756324147887767,
  0.6756386918586404,
  0.6677986315987698])

In [12]:
test = pd.read_csv("/home/tejsax/Downloads/test.csv")
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [13]:
test.drop(["PassengerId", "Name", "Ticket", "Fare", "Cabin"], axis = 1, inplace = True)
test

,Pclass,Sex,Age,SibSp,Parch,Embarked
0,3,male,34.5,0,0,Q
1,3,female,47.0,1,0,S
2,2,male,62.0,0,0,Q
3,3,male,27.0,0,0,S
4,3,female,22.0,1,1,S
...,...,...,...,...,...,...
413,3,male,NaN,0,0,S
414,1,female,39.0,0,0,C
415,3,male,38.5,0,0,S
416,3,male,NaN,0,0,S


In [14]:
test["Embrked"] = test["Embarked"].apply(femb)
test.drop(["Embarked"], axis = 1, inplace = True)
test

,Pclass,Sex,Age,SibSp,Parch,Embrked
0,3,male,34.5,0,0,2
1,3,female,47.0,1,0,0
2,2,male,62.0,0,0,2
3,3,male,27.0,0,0,0
4,3,female,22.0,1,1,0
...,...,...,...,...,...,...
413,3,male,NaN,0,0,0
414,1,female,39.0,0,0,1
415,3,male,38.5,0,0,0
416,3,male,NaN,0,0,0


In [15]:
test.fillna(test.median(), inplace = True)
test

,Pclass,Sex,Age,SibSp,Parch,Embrked
0,3,male,34.5,0,0,2
1,3,female,47.0,1,0,0
2,2,male,62.0,0,0,2
3,3,male,27.0,0,0,0
4,3,female,22.0,1,1,0
...,...,...,...,...,...,...
413,3,male,27.0,0,0,0
414,1,female,39.0,0,0,1
415,3,male,38.5,0,0,0
416,3,male,27.0,0,0,0


In [16]:
test["M1/F0"] = test["Sex"].apply(fsex)
test.drop(["Sex"], axis = 1, inplace = True)
test

,Pclass,Age,SibSp,Parch,Embrked,M1/F0
0,3,34.5,0,0,2,1
1,3,47.0,1,0,0,0
2,2,62.0,0,0,2,1
3,3,27.0,0,0,0,1
4,3,22.0,1,1,0,0
...,...,...,...,...,...,...
413,3,27.0,0,0,0,1
414,1,39.0,0,0,1,0
415,3,38.5,0,0,0,1
416,3,27.0,0,0,0,1


In [17]:
x_test = test.to_numpy()
x_test

array([[ 3. , 34.5,  0. ,  0. ,  2. ,  1. ],
       [ 3. , 47. ,  1. ,  0. ,  0. ,  0. ],
       [ 2. , 62. ,  0. ,  0. ,  2. ,  1. ],
       ...,
       [ 3. , 38.5,  0. ,  0. ,  0. ,  1. ],
       [ 3. , 27. ,  0. ,  0. ,  0. ,  1. ],
       [ 3. , 27. ,  1. ,  1. ,  1. ,  1. ]])

In [46]:
# Logistic Regression
clf_lr = LogisticRegression()

# grid search for the best possible parameters
grid = {"C": [10**i for i in range (-5, 5)], "max_iter": [10**i for i in range (1, 4)]}
gs_logistic = GridSearchCV(clf_lr, grid, cv=KFold(n_splits=10, shuffle=True, random_state=0))
gs_logistic.fit(x, y)
gs_logistic.best_estimator_

/home/tejsax/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/tejsax/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

/home/tejsax/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/tejsax/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

/home/tejsax/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/tejsax/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

/home/tejsax/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/tejsax/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

/home/tejsax/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/tejsax/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [51]:
# Random Forest
clf_rf = RandomForestClassifier(criterion="entropy", random_state=0)

# grid search for the best possible parameters
grid = {"n_estimators": [5, 10, 15, 20, 25], "max_depth": [10, 15, 20, 25, 30, 35]}
gs_rf = GridSearchCV(clf_rf, grid, cv=KFold(n_splits=10, shuffle=True, random_state=0))
gs_rf.fit(x, y)
gs_rf.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [54]:
# KNN
clf_knn = KNeighborsClassifier()

# grid search for the best possible parameters
grid = {"n_neighbors": [3, 5, 7, 9, 11]}
gs_knn = GridSearchCV(clf_knn, grid, cv=KFold(n_splits=10, shuffle=True, random_state=0))
gs_knn.fit(x, y)
gs_knn.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [59]:
# SVM
clf_svc = SVC()

# grid search for the best possible parameters
grid = {"C": [10**i for i in range (2, 5)], "gamma": [1e-3, 5e-4, 1e-4]}
gs_svc = GridSearchCV(clf_svc, grid, cv=KFold(n_splits=10, shuffle=True, random_state=0))
gs_svc.fit(x, y)
gs_svc.best_estimator_

SVC(C=10000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [32]:
gs_logistic.best_score_

0.8069288389513108

In [42]:
gs_rf.best_score_

0.8148689138576779

In [55]:
gs_knn.best_score_

0.7766541822721599

In [60]:
gs_svc.best_score_

0.8215355805243444

In [62]:
# TRAINING with best parameters and algorithm

clf = SVC(C=10000, gamma=0.001)
clf.fit(x, y)

SVC(C=10000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [63]:
y_pred = clf.predict(x_test)
y_pred

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [64]:
savetxt('kaggle_prediction.csv', y_pred)